In [1]:
import numpy as np
import pandas as pd


In [2]:
d = pd.read_csv("spam_ham_dataset.csv")

In [3]:
d.head(5)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [4]:
d.shape

(5171, 4)

In [5]:
d.notnull().sum()

Unnamed: 0    5171
label         5171
text          5171
label_num     5171
dtype: int64

In [6]:
#taking only the text and label_num
df = d[["text","label_num"]]

In [7]:
x = df["text"]
y = df["label_num"]
y

0       0
1       0
2       0
3       1
4       0
       ..
5166    0
5167    0
5168    0
5169    0
5170    1
Name: label_num, Length: 5171, dtype: int64

In [8]:
#split data

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2 , random_state =42 )

<h1>TEXT PREPROCESSING</h1>

tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")<br>
tokenizer.fit_on_texts(["hello world", "hello spam"]<br>)

print(tokenizer.word_inde<br>x)V>': 1, 'hello': 2, 'world': 3, 'spam<br>: 4}

seq = tokenizer.texts_to_sequences(["hello s<br>pam"])
print<br>(2,4[[2, 4]]

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer  #tokeniser convert raw text -> integers
from tensorflow.keras.preprocessing.sequence import pad_sequences #makes all sequences the same length by adding padding (usually zeros).

#limiting vocab_size for memory
vocab_size = 10000
maxlen = 20 #maximum token per email

tokenizer = Tokenizer(num_words = vocab_size, oov_token ="<OOV>")  #oov means out-of-Vocabulary by setting the token the tokenizer create
                                                                   # new spcl token for unseen words
                                                                   # without oov the tokenizer just dropa unseen words

tokenizer.fit_on_texts(x_train)

#convert text -> sequence
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

#padding
x_train_pad = pad_sequences(x_train_seq, maxlen=maxlen, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=maxlen, padding='post')



<h1>BUILDING THE LSTM MODEL</h1>

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(input_dim = vocab_size, output_dim = 64, input_length = maxlen),
    LSTM(64, return_sequences = False),
    Dropout(0.5),
    Dense(32, activation = "relu"),
    Dense(1, activation = "sigmoid")
])



model.compile(loss="binary_crossentropy",   
              optimizer="adam",           
              metrics=['accuracy'])  

model.summary()

C:\Users\saani\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
#fitting the model
history  = model.fit(x_train_pad, y_train, 
                    epochs = 4, batch_size = 64,
                    validation_data=(x_test_pad, y_test))

Epoch 1/4
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9998 - loss: 0.0030 - val_accuracy: 0.9565 - val_loss: 0.1612
Epoch 2/4
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9995 - loss: 0.0041 - val_accuracy: 0.9536 - val_loss: 0.1866
Epoch 3/4
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9984 - loss: 0.0050 - val_accuracy: 0.9623 - val_loss: 0.1460
Epoch 4/4
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9993 - loss: 0.0058 - val_accuracy: 0.9536 - val_loss: 0.2831


<h3 color = "green">MODEL ACCURACY : 96.71%</h3>

In [13]:
from sklearn.metrics import confusion_matrix, classification_report

loss, acc = model.evaluate(x_test_pad, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")

y_pred = (model.predict(x_test_pad) > 0.5).astype("int32")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9542 - loss: 0.2757  
Test Accuracy: 95.36%
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
[[728  14]
 [ 34 259]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       742
           1       0.95      0.88      0.92       293

    accuracy                           0.95      1035
   macro avg       0.95      0.93      0.94      1035
weighted avg       0.95      0.95      0.95      1035



<h1>TESTING WITH UNSEEN DATA</h1>

In [14]:
sample_emails = [
    #"Subject: Win a FREE iPhone! Click here now.",
    "Subject : HO HO HO WIN FREE OFFER! TODAY!",
    "Subject: Meeting schedule for tomorrow"
]

seqs = tokenizer.texts_to_sequences(sample_emails)
padded = pad_sequences(seqs, maxlen=maxlen, padding='post')
preds = model.predict(padded)

for email, pred in zip(sample_emails, preds):
    print(f"{email} -> {'SPAM' if pred > 0.5 else 'HAM'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Subject : HO HO HO WIN FREE OFFER! TODAY! -> SPAM
Subject: Meeting schedule for tomorrow -> HAM


In [21]:
import os
os.makedirs("saved-model", exist_ok=True)

In [22]:
model.save('saved-model/spam_model.keras') #saving the model